In [1]:
import pickle
import pandas as pd
import numpy
import re
import os
import numpy as np
import gensim
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from gensim.models import Doc2Vec
import warnings
import mplcursors
import matplotlib.pyplot as plt
import mplcursors
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import PyQt6
import PySide6
# import PyQt5
# import PySide2
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


### K Means is used to find the Nearest Documents and Document Clusters

1. After learning K-means, we have videos clustered by similarity according to Doc2Vec.


2. We can recommend videos within the closest cluster if that video is queried. 

-If we get the vector for the new video we can do kmeans.predict but we probably have to retrain the pca

-Or, as a weaker substitute we can do a for loop over the k means center videos and do a cosine similarity between a query
and those centers to find which cluster is relevant to recommend to the user.

3. As an additional step we can find the cosine similarity between a video and a recommended video or its similarity within a large pretrained embedding like GloVE to make sure the recommendations in that cluster are relevant.

In [2]:
path = '../data_cleaning/data/video_transcripts_clean.csv'
clean_transcripts = pd.read_csv(path)
clean_transcripts.head(1)

,Unnamed: 0,Title,Video ID,Published At,Keyword,Likes,Comments,Views,Is_Generated,Transcript_Blob,Transcript_Clean,Title_Clean
0,0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,2022-08-23,tech,3407.0,672.0,135612.0,True,[Music] this is your tech news briefing for t...,music tech news brief tuesday august 23rd im z...,appl pay kill physic wallet eight year tech ne...


In [3]:
clean_transcripts['Title_and_Transcript_Clean'] = clean_transcripts['Title_Clean'] + ' ' + clean_transcripts['Transcript_Clean']
clean_titles_list = clean_transcripts['Title'].tolist()

In [4]:
stop_words = set(stopwords.words('english'))
has_stopwords = clean_transcripts['Title_and_Transcript_Clean'].apply(lambda x: isinstance(x, str) and any(word.lower() in stop_words for word in x.split()))

# Check if any values in the column contain stopwords
if has_stopwords.any():
    print("The column contains stopwords.")
else:
    print("The column does not contain stopwords.")


print( len(stop_words))

The column contains stopwords.
179


Remove Stop Words

In [5]:
#clean_transcripts['Transcript_Clean'] = clean_transcripts['Transcript_Clean'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
#clean_transcripts['Title_Clean'] = clean_transcripts['Title_Clean'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
#clean_transcripts['Title_and_Transcript_Clean'] = clean_transcripts['Title_and_Transcript_Clean'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
clean_transcripts['Title_and_Transcript_Clean'] = clean_transcripts['Title_and_Transcript_Clean'].apply(lambda x: ' '.join([word for word in str(x).split() if word.lower() not in stop_words]))


has_stopwords = clean_transcripts['Title_and_Transcript_Clean'].apply(lambda x: isinstance(x, str) and any(word.lower() in stop_words for word in x.split()))

# Check if any values in the column contain stopwords
if has_stopwords.any():
    print("The column contains stopwords.")
    

In [6]:
identified_stopwords = clean_transcripts['Title_and_Transcript_Clean'].apply( lambda x:[word.lower() for word in x.split() if isinstance(x, str) and word.lower() in stop_words])

for wordlist in identified_stopwords:
    if len(wordlist) > 0:
        print( wordlist )

### Doc2Vec

In [7]:
#Select the Data to train on 

#I suspect that the Title is the most important - but is the transcript better?

#all_content_train = clean_transcripts['Transcript_Blob'].astype(str).tolist()
#all_content_train = clean_transcripts['Transcript_Clean'].astype(str).tolist()
#all_content_train = clean_transcripts['Title_Clean'].astype(str).tolist()
all_content_train = clean_transcripts['Title_and_Transcript_Clean'].astype(str).tolist()

In [8]:
tagged_documents = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(all_content_train)]
d2v_model = Doc2Vec(tagged_documents, vector_size=100, window=10, min_count=500, workers=7, dm=1, alpha=0.025, min_alpha=0.001)
d2v_model.train(tagged_documents, total_examples=d2v_model.corpus_count, epochs=10, start_alpha=0.002, end_alpha=-0.016)

### KMeans

In [9]:
from collections import Counter
#https://medium.com/@ermolushka/text-clusterization-using-python-and-doc2vec-8c499668fa61
def fit_kmeans(n_clusters):
    kmeans_model = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=10000)
    X = kmeans_model.fit(d2v_model.dv.vectors)
    labels = kmeans_model.labels_.tolist()
    #l = kmeans_model.fit_predict(d2v_model.dv.vectors) #does nothing?
    pca = PCA(n_components=10,random_state=42).fit(d2v_model.dv.vectors)
    datapoint = pca.transform(d2v_model.dv.vectors)
    value_counts = Counter(labels)
    return datapoint, value_counts
datapoint, value_counts = fit_kmeans(20)

This dictionary shows the count videos in each kmeans cluster

In [11]:
print(value_counts)

Counter({2: 451, 16: 351, 9: 166, 15: 135, 4: 122, 1: 112, 6: 111, 3: 88, 18: 70, 11: 63, 17: 61, 10: 44, 8: 29, 7: 17, 0: 16, 13: 13, 19: 13, 12: 11, 14: 4, 5: 4})


We can find the Clusters with the Least Points which may be more informative

In [12]:
sorted_keys = sorted(value_counts, key=lambda k: value_counts[k])
print(f"The Cluster Numbers from Least to Most Points {sorted_keys})")

The Cluster Numbers from Least to Most Points [14, 5, 12, 13, 19, 0, 7, 8, 10, 17, 11, 18, 3, 6, 1, 4, 15, 9, 16, 2])


In [13]:
def print_group_examples(group_num, total_examples):
    print(f"Here is kmeans group {group_num}")
    print(" ")
    found = 0
    for count in range(len(clean_titles_list)):
        if labels[count]==group_num:
            print(clean_titles_list[count])
            found+=1
        if found ==total_examples:
            break
print_group_examples(sorted_keys[0],5)

Here is kmeans group 14
 


NameError: name 'labels' is not defined

In [ ]:
print_group_examples(sorted_keys[1],5)

Here is kmeans group 13
 
Taste Expert Answers Questions From Twitter | Tech Support | WIRED
Audiologist Answers Hearing Questions From Twitter | Tech Support | WIRED
James Dyson Answers Design Questions From Twitter | Tech Support | WIRED
Veterinarian Answers Pet Questions From Twitter | Tech Support | WIRED
What Is The Best Position For Descending? | GCN Tech Clinic


Plot K Means -- Though manual search above is recommended

In [ ]:
%matplotlib qt
def plot_kmeans(n_clusters):
    try:
        fig, ax = plt.subplots()
        label1 = [
        "#FF0000", "#00FF00", "#0000FF", "#FFFF00", "#00FFFF", "#FF00FF",  # Primary colors and cyan, magenta, and yellow
        "#800000", "#008000", "#000080", "#808000", "#008080", "#800080",  # Darker shades of the primary colors and their combinations
        "#FFA500", "#FFFF99", "#FF99CC", "#CC99FF", "#99CCFF", "#99FFCC",  # Orange, light yellow, light pink, light purple, light blue-green combinations
        "#FFC0CB", "#800080", "#008080", "#00FF00", "#FF0000", "#0000FF",  # Pink, purple, teal, lime, red, blue
        "#000000", "#808080", "#C0C0C0", "#FFFFFF", "#F5F5DC", "#A52A2A",  # Black, gray shades, white, beige, brown
        "#F0FFFF", "#F0F8FF", "#FAEBD7", "#7FFFD4", "#F5F5F5", "#FFD700",  # Azure, alice blue, antique white, aquamarine, white smoke, gold
        "#D2B48C", "#ADFF2F", "#FF69B4", "#4B0082", "#7FFF00", "#800000"]  # Tan, green-yellow, hot pink, indigo, chartreuse, maroon]    label1 = labels1[:n_clusters]
        color = [label1[i] for i in labels]

        scatter = ax.scatter(datapoint[:, 0], datapoint[:, 1], c=color)
        # Create the cursor object
        cursor = mplcursors.cursor(scatter)
        # Define the tooltip content
        @cursor.connect("add")
        def on_add(sel):
            index = sel.target.index
            label = labels[index]
            transcript = clean_titles_list[index]  # Get the transcript based on the index
            sel.annotation.set_text(f'Label: {label}\nTranscript: {transcript}')

        # Add labels to each data point
        for i, label in enumerate(labels):
            ax.text(datapoint[i, 0], datapoint[i, 1], label, ha='center', va='center')

        plt.show()
    except AttributeError:
        pass

#plot_kmeans(n_clusters)


Find Closest Cluster of a new Query of Words

In [ ]:
from nltk.tokenize import word_tokenize

query = "SAT Math Prep - the No Calculator Practice Test 1"
stop_words = set(stopwords.words('english'))
query = query.split()
query = [word for word in query if word.lower() not in stop_words]
print(query)

query = [' '.join(query)]
print(query)
#tagged_documents = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(all_content_train)]
#d2v_model = Doc2Vec(tagged_documents, vector_size=100, window=10, min_count=500, workers=7, dm=1, alpha=0.025, min_alpha=0.001)
#d2v_model.train(tagged_documents, total_examples=d2v_model.corpus_count, epochs=10, start_alpha=0.002, end_alpha=-0.016)
inferred_vectors = d2v_model.infer_vector(query).reshape(1, -1)
#datapoint = pca.transform(new_vec)
d2v_model.docvecs.add_documents(new_documents, inferred_vectors) 
#closest_cluster = int(kmeans_model.predict(new_vec))
#print(type(closest_cluster),closest_cluster)
print_group_examples(sorted_keys[closest_cluster],5)

['SAT', 'Math', 'Prep', '-', 'Calculator', 'Practice', 'Test', '1']
['SAT Math Prep - Calculator Practice Test 1']


NameError: name 'd2v_model' is not defined

# IGNORE Experimental Section Below 

DBSCAN is an alternative to Kmeans that may learn more complex grouping shapes

In [ ]:


%matplotlib qt

dbscan_model = DBSCAN(eps=0.7, min_samples=15)
X = dbscan_model.fit(d2v_model.dv.vectors)

labels = dbscan_model.labels_.tolist()
pca = PCA(n_components=5).fit(d2v_model.dv.vectors)
datapoint = pca.transform(d2v_model.dv.vectors)

fig, ax = plt.subplots()

label1 = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF', '#800080', '#FFA500']
color = [label1[i] for i in labels]

scatter = ax.scatter(datapoint[:, 0], datapoint[:, 1], c=color)

# Create the cursor object
cursor = mplcursors.cursor(scatter)

# Define the tooltip content
@cursor.connect("add")
def on_add(sel):
    index = sel.target.index
    label = labels[index]
    transcript = clean_titles_list[index]  # Get the transcript based on the index
    sel.annotation.set_text(f'Label: {label}\nTranscript: {transcript}')

# Add labels to each data point
for i, label in enumerate(labels):
    ax.text(datapoint[i, 0], datapoint[i, 1], label, ha='center', va='center')

plt.show()


In [ ]:

# Create 30 evenly spaced values between 0.1 and 1.0 for eps
eps_values = np.linspace(0.1, 1.0, 30)

# Create 30 evenly spaced values between 5 and 50 for min_samples (rounded to nearest integer)
min_samples_values = np.round(np.linspace(5, 50, 30)).astype(int)

# Combine these into pairs
param_pairs = list(zip(eps_values, min_samples_values))


In [ ]:


%matplotlib qt

# Define the parameter pairs to try
#param_pairs = [(0.5, 5), (0.3, 10), (0.7, 3), (0.2, 10), (1.0, 5)]
try:
    more_found = False
    for eps, min_samples in param_pairs:
        # Fit DBSCAN model
        dbscan_model = DBSCAN(eps=eps, min_samples=min_samples)
        X = dbscan_model.fit(d2v_model.dv.vectors)

        # Get labels
        labels = dbscan_model.labels_.tolist()

        # Check if we have more than one cluster (excluding noise, represented by -1)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        
        if n_clusters > 1:
            more_found = True
            print(f"Found {n_clusters} clusters with eps={eps}, min_samples={min_samples}")
            break
        else:
            pass
    if not more_found:
        print("only one cluster found")
    # Continue with the script if more than one cluster found
    pca = PCA(n_components=2).fit(d2v_model.dv.vectors)
    datapoint = pca.transform(d2v_model.dv.vectors)

    fig, ax = plt.subplots()

    label1 = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF', '#800080', '#FFA500']
    color = [label1[i] for i in labels]

    scatter = ax.scatter(datapoint[:, 0], datapoint[:, 1], c=color)

    # Create the cursor object
    cursor = mplcursors.cursor(scatter)

    # Define the tooltip content
    @cursor.connect("add")
    def on_add(sel):
        index = sel.target.index
        label = labels[index]
        transcript = clean_titles_list[index]  # Get the transcript based on the index
        sel.annotation.set_text(f'Label: {label}\nTranscript: {transcript}')

    # Add labels to each data point
    for i, label in enumerate(labels):
        ax.text(datapoint[i, 0], datapoint[i, 1], label, ha='center', va='center')

    plt.show()
except AttributeError:
    pass



Found 5 clusters with eps=0.1, min_samples=5
